In [1]:
-- load images, scale, store in tensor


-- make labels


--Load the AlexNet model
alexnet = torch.load('alexnetowtbn_epoch55_cpu.t7')
alexnet:evaluate()

-- Pre-process the image channel by channel.
function preprocess(im)
    local output_image = image.scale(im:clone(), 224, 224)
    for i = 1, 3 do -- channels
        output_image[{{i},{},{}}]:add(-meanStd.mean[i])
        output_image[{{i},{},{}}]:div(meanStd.std[i])
    end
    return output_image
end

--Get fc6 output given a model and input image
function fc6output(model, im)
    
    

--Build linear model (takes 2(4096))
--run "linearmodel:forward({x, y})", x and y are fc6outputs
function build_linear_model():
    local linearmodel = nn.Sequential()
    linearmodel:add(nn.PairwiseDistance(2))
    linearmodel:add(nn.Linear(4096,512))
    linearmodel:add(nn.Linear(512, 1))
    return linearmodel

function predict(model, img_a, img_b):
    fc6_a = fc6output(alexnet, img_a)
    fc6_b = fc6output(alexnet, img_b)
    model:forward({fc6_b})
    
    


[string "-- load images, scale, store in tensor..."]:28: unexpected symbol near ':': 

In [14]:
require 'nn'

--Load the AlexNet model
meanStd = torch.load('alexnetowtbn_meanStd.t7')
alexnet = torch.load('alexnetowtbn_epoch55_cpu.t7')
alexnet:evaluate()


-- Pre-process the image channel by channel.
function preprocess(im)
    local output_image = image.scale(im:clone(), 224, 224)
    for i = 1, 3 do -- channels
        output_image[{{i},{},{}}]:add(-meanStd.mean[i])
        output_image[{{i},{},{}}]:div(meanStd.std[i])
    end
    return output_image
end



In [55]:
function extract_fc6(input_image)
    input_image = image.scale(input_image, 224, 224)
    -- itorch.image(input_image)
    input_image = preprocess(input_image):view(1, 3, 224, 224)
    local predictions = alexnet:forward(input_image)
    -- print(predictions)
    return alexnet:get(2):get(3).output
end


linearmodel = nn.Sequential()
linearmodel:add(nn.Linear(8192,512))
linearmodel:add(nn.Linear(512, 1))



In [ ]:
-- TODO: need to change BCECriterion
local criterion = nn.BCECriterion() -- Negative log-likelihood criterion.
-- params is a flat vector with the concatenation of all the parameters inside model.
-- gradParams is a flat vector with the concatenation of all the gradients of parameters inside the model.
-- These two variables also merely point to the internal individual parameters in each layer of the module.

function trainModel(model, opt, features, preprocessFn)
    -- Get all the parameters (and gradients) of the model in a single vector.
    local params, gradParams = model:getParameters()

    local opt = opt or {}
    local batchSize = opt.batchSize or 64  -- The bigger the batch size the most accurate the gradients.
    local learningRate = opt.learningRate or 0.001  -- This is the learning rate parameter often referred to as lambda.
    local momentumRate = opt.momentumRate or 0.9
    local numEpochs = opt.numEpochs or 3
    local velocityParams = torch.zeros(gradParams:size())
    local train_features, val_features
    -- TODO: need to remove those!!
    if preprocessFn then
        train_features = trainset.data:float():div(255)
        val_features = valset.data:float():div(255)
    else
        train_features = (features and features.train_features) or trainset.normdata
        val_features = (features and features.val_features) or valset.normdata
    end
    -- Go over the training data this number of times.
    for epoch = 1, numEpochs do
        local sum_loss = 0
        local correct = 0
        
        -- Run over the training set samples.
        -- set internal var Train=True so that layers like Dropout behave correctly. Also see :evaluate()
        model:training()
        for i = 1, trainset.normdata:size(1) / batchSize do
            
            -- 1. Sample a batch.
            local inputs
            if preprocessFn then
                inputs = torch.Tensor(batchSize, 3, 224, 224)
            else
                inputs = (features and torch.Tensor(batchSize, 4096)) or torch.Tensor(batchSize, 3, 32, 32)
            end
            local labels = torch.Tensor(batchSize)
            for bi = 1, batchSize do
                local rand_id = torch.random(1, train_features:size(1))
                if preprocessFn then
                    inputs[bi] = preprocessFn(train_features[rand_id])
                else
                    inputs[bi] = train_features[rand_id]
                end
                labels[bi] = trainset.label[rand_id]
            end
            -- 2. Perform the forward pass (prediction mode).
            local predictions = model:forward(inputs)
            
            -- 3. Evaluate results.
            for i = 1, predictions:size(1) do
                local _, predicted_label = predictions[i]:max(1)
                if predicted_label[1] == labels[i] then correct = correct + 1 end
            end
            sum_loss = sum_loss + criterion:forward(predictions, labels)
            
            -- TODO: need to figure out how to change this to accomandate the BCECriterion
            -- 4. Perform the backward pass (compute derivatives).
            -- This zeroes-out all the parameters inside the model pointed by variable params.
            model:zeroGradParameters()
            -- This internally computes the gradients with respect to the parameters pointed by gradParams.
            local gradPredictions = criterion:backward(predictions, labels)
            model:backward(inputs, gradPredictions)

            -- 5. Perform the SGD update.
            velocityParams:mul(momentumRate)
            velocityParams:add(learningRate, gradParams)
            params:add(-1, velocityParams)

            if i % 100 == 0 then  -- Print this every five thousand iterations.
                print(('train epoch=%d, iteration=%d, avg-loss=%.6f, avg-accuracy = %.2f')
                    :format(epoch, i, sum_loss / i, correct / (i * batchSize)))
            end
        end

        -- Run over the validation set for evaluation.
        local validation_accuracy = 0
        local nBatches = val_features:size(1) / batchSize
        model:evaluate()
        for i = 1, nBatches do
            
            -- 1. Sample a batch.
            if preprocessFn then
                inputs = torch.Tensor(batchSize, 3, 224, 224)
            else
                inputs = (features and torch.Tensor(batchSize, 4096)) or torch.Tensor(batchSize, 3, 32, 32)
            end
            local labels = torch.Tensor(batchSize)
            for bi = 1, batchSize do
                local rand_id = torch.random(1, val_features:size(1))
                if preprocessFn then
                    inputs[bi] = preprocessFn(val_features[rand_id])
                else
                    inputs[bi] = val_features[rand_id]
                end
                labels[bi] = valset.label[rand_id]
            end

            -- 2. Perform the forward pass (prediction mode).
            local predictions = model:forward(inputs)
            
            -- 3. evaluate results.
            for i = 1, predictions:size(1) do
                local _, predicted_label = predictions[i]:max(1)
                if predicted_label[1] == labels[i] then validation_accuracy = validation_accuracy + 1 end
            end
        end
        validation_accuracy = validation_accuracy / (nBatches * batchSize)
        print(('\nvalidation accuracy at epoch = %d is %.4f'):format(epoch, validation_accuracy))
    end
end

In [53]:
local input_image1 = extract_fc6(image.load('test.jpg', 3, 'float')):double()
local input_image2 = extract_fc6(image.load('test2.jpg', 3, 'float')):double()
local cat_input_image = torch.cat(input_image1, input_image2)
local response = linearmodel:forward(cat_input_image)
print(response)
-- print(extract_fc6(input_image))

 0.1189
[torch.DoubleTensor of size 1x1]

